In [1]:
!pip install kaggle

Importing dependencies

In [2]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data collection through kaggle API

In [3]:
kaggle_dictionary = json.load(open('kaggle.json'))
os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionary['key']

In [4]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 97% 25.0M/25.7M [00:02<00:00, 20.2MB/s]
100% 25.7M/25.7M [00:02<00:00, 12.0MB/s]


In [5]:
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zipObj:
  zipObj.extractall()

Loading dataset

In [6]:
data = pd.read_csv('/content/IMDB Dataset.csv')

In [7]:
data.shape

(50000, 2)

In [8]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [9]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [10]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [11]:
data.replace({'sentiment': {'positive': 1, 'negative': 0}}, inplace=True)

<ipython-input-11-d53fdcf96856>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({'sentiment': {'positive': 1, 'negative': 0}}, inplace=True)


In [12]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [13]:
data['sentiment'].value_counts()

,count
sentiment,
1,25000
0,25000


In [14]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [15]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


Data preprocessing

In [16]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])

In [17]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=200)

In [18]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [19]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [20]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

In [21]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [22]:
print(Y_test)

33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


Building LSTM Model

In [23]:
model = Sequential()
model.add(Embedding(input_dim=500, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [24]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Training the model

In [26]:
model.fit(X_train, Y_train, batch_size=64, epochs=5, validation_data=(X_test, Y_test), validation_split=0.2)

Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 239s 370ms/step - accuracy: 0.6902 - loss: 0.5792 - val_accuracy: 0.7933 - val_loss: 0.4478
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 233s 372ms/step - accuracy: 0.7817 - loss: 0.4725 - val_accuracy: 0.8334 - val_loss: 0.3809
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 261s 371ms/step - accuracy: 0.8078 - loss: 0.4265 - val_accuracy: 0.8204 - val_loss: 0.4185
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 261s 370ms/step - accuracy: 0.8191 - loss: 0.4102 - val_accuracy: 0.8267 - val_loss: 0.3931
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 263s 372ms/step - accuracy: 0.8177 - loss: 0.3993 - val_accuracy: 0.8385 - val_loss: 0.3720


Model evaluation

In [27]:
loss, accuracy = model.evaluate(X_test, Y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

313/313 ━━━━━━━━━━━━━━━━━━━━ 36s 114ms/step - accuracy: 0.8344 - loss: 0.3803
Test Loss: 0.3719668984413147
Test Accuracy: 0.8385000228881836


Build predictive system

In [28]:
def predictSentiment(review):
  review = tokenizer.texts_to_sequences([review])
  review = pad_sequences(review, maxlen=200)
  prediction = model.predict(review)
  if prediction > 0.5:
    print('Positive Review')
    print('Probability:', prediction[0][0])
  else:
    print('Negative Review')
    print('Probability:', 1 - prediction[0][0])

In [29]:
new_review = "I loved this movie. It was amazing!"
predictSentiment(new_review)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 451ms/step
Positive Review
Probability: 0.9703709


In [30]:
new_review = "Movie was not upto mark!"
predictSentiment(new_review)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
Negative Review
Probability: 0.64912348985672


In [32]:
new_review = "Movie was good but cannot be appreciated."
predictSentiment(new_review)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
Positive Review
Probability: 0.60365444
